In [65]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 自定义模块
from torchLearning import *

In [66]:
import random
import torch

def data_iter(batch_size, features, labels):
    """
    数据切分函数
    
    :param batch_size: 每个子数据集包含多少数据
    :param featurs: 输入的特征张量
    :param labels：输入的标签张量
    :return l：包含batch_size个列表，每个列表切分后的特征和标签所组成 
    """
    # 获取总样本数量
    num_examples = len(features)
    
    # 创建从0到num_examples-1的索引列表
    indices = list(range(num_examples))
    
    # 随机打乱索引顺序，实现数据的随机化
    random.shuffle(indices)
    
    # 初始化空列表用于存储所有批量数据
    l = []
    
    # 以batch_size为步长遍历所有样本索引
    for i in range(0, num_examples, batch_size):
        # i 是当前批量在总数据中的起始位置
        # 获取当前批量的索引范围，处理最后一个批量可能不足的情况
        j = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        
        # 使用索引选择对应的特征和标签数据，并作为一个批量添加到结果列表
        l.append([torch.index_select(features, 0, j), torch.index_select(labels, 0, j)])
    
    # 返回包含所有批量的列表
    return l
def tensorGenReg(num_examples=1000, w=[2, -1, 1], bias=True, delta=0.01, deg=1):
    """
    生成带多项式特征的人工合成数据集（可自定义噪声和多项式阶数）
    
    参数：
        num_examples (int): 样本数量（默认1000）
        w (list/tensor): 真实权重系数（例如 [2,-1,1] 表示 2*x1 -1*x2 +1*x1^2）
        bias (bool): 是否包含偏置项（默认True）
        delta (float): 噪声标准差（默认0.01）
        deg (int): 多项式阶数（默认1，即线性）
        
    返回：
        features (Tensor): 生成的特征矩阵 (num_examples, num_features)
        labels (Tensor): 生成的标签 (num_examples, 1)
    """
    # 当包含偏置项时
    if bias==True:
        # 输入特征数量 = 权重数量-1（最后一个权重是偏置项）
        num_inputs=len(w)-1
        # 生成随机特征矩阵（标准正态分布）
        features_ture=torch.randn(num_examples,num_inputs)
        # 提取权重（排除最后一个偏置项）并转为列向量
        w_true=torch.tensor(w[:-1]).reshape(-1,1).float()
        # 提取偏置项（最后一个权重）
        b_true=torch.tensor(w[-1]).float()
        
        # 计算真实标签（带多项式）
        if num_inputs==1:  # 单特征情况
            labels_true=torch.pow(features_ture,deg)*w_true+b_true  # y = x^deg * w + b
        else:  # 多特征情况
            labels_true=torch.mm(torch.pow(features_ture,deg),w_true)+b_true  # y = X^deg @ w + b
        
        # 在特征矩阵右侧添加一列全1（用于偏置项）
        features=torch.cat((features_ture,torch.ones(size=[len(features_ture),1])),dim=1)
        # 在真实标签上添加高斯噪声
        labels=labels_true+torch.randn(size=labels_true.shape)*delta
    
    # 当不包含偏置项时
    else:
        # 输入特征数量 = 权重数量（所有权重都用于特征）
        num_inputs=len(w)
        # 生成随机特征矩阵（标准正态分布）
        features_ture=torch.randn(num_examples,num_inputs)
        # 所有权重都转为列向量
        w_true=torch.tensor(w).reshape(-1,1).float()
        
        # 计算真实标签（带多项式）
        if num_inputs==1:  # 单特征情况
            labels_true=torch.pow(features_ture,deg)*w_true  # y = x^deg * w
        else:  # 多特征情况
            labels_true=torch.mm(torch.pow(features_ture,deg),w_true )# y = X^deg @ w
        
        # 特征矩阵保持不变（不添加偏置列）
        features=features_ture
        # 在真实标签上添加高斯噪声
        labels=labels_true+torch.randn(size=labels_true.shape)*delta
    
    return features, labels 
            

In [67]:
# 0.确定数据集
features, labels =tensorGenReg()

# 1. 选择模型
def linreg(X, w):
    return torch.mm(X, w)  # 线性回归模型：X × w

# 2. 目标函数-损失函数
def squared_loss(y_hat, y):
    num_ = y.numel()  # 样本数量
    sse = torch.sum((y_hat.reshape(-1, 1) - y.reshape(-1, 1)) ** 2)  # 平方误差和
    return sse / num_  # 均方误差

# 3. 优化算法
def sgd(params, lr):
    params.data -= lr * params.grad  # 参数更新
    params.grad.zero_()  # 梯度清零

# 设置随机数种子
torch.manual_seed(420)

# 初始化参数
batch_size = 10
lr = 0.03
num_epochs = 3
w = torch.zeros(3, 1, requires_grad=True)  # 3个权重（2特征+1截距）

net = linreg
loss = squared_loss

# 模型训练过程（修正后的）
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        # ✅ 关键的三步训练流程：
        l = loss(net(X, w), y)  # 前向传播：计算损失
        l.backward()            # 反向传播：计算梯度
        sgd(w, lr)              # 参数更新：优化权重
    
    # 每个epoch结束后评估整体性能
    train_l = loss(net(features, w), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l))

epoch 1, loss 0.000126
epoch 2, loss 0.000101
epoch 3, loss 0.000102


使用summaryWriter

In [68]:
writer = SummaryWriter(log_dir='reg_loss')
# 初始化核心参数
batch_size = 10                                # 每一个小批的数量
lr = 0.03                                      # 学习率
num_epochs = 3                                 # 训练过程遍历几次数据
w = torch.zeros(3, 1, requires_grad = True)    # 随机设置初始权重

# 参与训练的模型方程
net = linreg                                   # 使用回归方程
loss = squared_loss                            # 均方误差的一半作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w), y)
        l.backward()
        sgd(w, lr)
    train_l = loss(net(features, w), labels)
    writer.add_scalar('mul', train_l, epoch)

快速实现

In [69]:
 #1.核心参数
batch_size = 10                                # 设置每个训练批次包含10个样本
lr = 0.03                                      # 设置学习率为0.03，控制参数更新步长
num_epochs = 3                                 # 设置训练过程遍历整个数据集3次

#2.数据准备
# 设随机数种子
torch.manual_seed(420)                         # 设置随机种子为420，确保实验结果可重现

# 创建数据集
features, labels = tensorGenReg()              # 调用数据生成函数创建特征和标签
features = features[:, :-1]                    # 剔除特征矩阵最后一列（全1的截距项）
data = TensorDataset(features, labels)         # 将特征和标签封装成PyTorch数据集对象
batchData = DataLoader(data, batch_size = batch_size, shuffle = True)  # 创建数据加载器，设置批量大小和打乱顺序

#3.定义模型
class LR(nn.Module):                           # 定义线性回归类，继承自nn.Module
    def __init__(self, in_features=2, out_features=1):  # 初始化方法：定义模型的层结构
        super(LR, self).__init__()             # 调用父类初始化方法
        self.linear = nn.Linear(in_features, out_features)  # 创建线性层，输入2特征，输出1个值
        
    def forward(self, x):                      # 定义前向传播方法：输入如何通过各层得到输出
        out = self.linear(x)                   # 输入数据通过线性层计算输出
        return out                             # 返回预测结果

# 实例化模型
LR_model = LR()                                # 创建线性回归模型的实例

#4.损失函数
criterion = nn.MSELoss()                       # 定义均方误差损失函数

#5.优化方法
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)  # 创建随机梯度下降优化器，传入模型参数和学习率

#6.模型训练
def fit(net, criterion, optimizer, batchdata, epochs):  # 定义训练函数，接收模型、损失函数、优化器、数据和训练轮数
    for epoch in range(epochs):                # 外层循环：遍历训练轮数
        for X, y in batchdata:                 # 内层循环：遍历每个批次的数据
            yhat = net.forward(X)              # 前向传播：计算当前批次的预测值
            loss = criterion(yhat, y)          # 计算损失：比较预测值和真实值的差异
            optimizer.zero_grad()              # 梯度清零：清空上一轮的梯度信息，防止累积
            loss.backward()                    # 反向传播：计算损失函数关于模型参数的梯度
            optimizer.step()                   # 参数更新：根据梯度方向调整模型参数
        writer.add_scalar('loss', loss, global_step=epoch)  # 记录当前epoch的损失值到TensorBoard（这行代码不完整）

In [70]:
# 设置随机数种子
torch.manual_seed(420)   

fit(net = LR_model, 
    criterion = criterion, 
    optimizer = optimizer, 
    batchdata = batchData, 
    epochs = num_epochs)
LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [71]:
# 查看模型参数
list(LR_model.parameters())

[Parameter containing:
 tensor([[ 2.0006, -1.0000]], requires_grad=True),
 Parameter containing:
 tensor([1.0008], requires_grad=True)]

In [72]:
# 计算MSE
criterion(LR_model(features), labels)

tensor(0.0001, grad_fn=<MseLossBackward0>)

In [73]:
writer.add_graph(LR_model, (features,))